In [10]:
import pandas as pd
import numpy as np

In [11]:
%%time
twitter = pd.read_csv("preprocessed_data.csv")    #normal one used before
#twitter = pd.read_csv("sorted_preprocessed_data.csv")  #with lowest emotion scores

Wall time: 546 ms


In [12]:
pd.set_option('display.max_rows', 1000)
twitter

,_score,tweet_id,text,identification,emotion,Category
0,299,0x2e7caf,my sunday lineup girlstrip power 💪🏾😎,train,fear,1
1,784,0x2eff99,therell never be a time than now to what you...,train,trust,2
2,358,0x3813a3,taluuluu riiiiiiiiiiiiiiiiight 🙄,train,anger,3
3,1021,0x362eaa,never give up on your dream me to make your ...,train,anticipation,4
4,367,0x286e23,pearlharper youre the most responsible adult i...,train,surprise,5
...,...,...,...,...,...,...
317658,887,0x27028e,i better go eat something because i havent eat...,train,anticipation,4
317659,574,0x30ffbb,russia had no bearing on electionpresident did...,train,anger,3
317660,91,0x29ccd4,artziiflower beingsalmankhan that meansnow u d...,train,sadness,8
317661,532,0x345ac6,talkmaster now in greenville schigh of enjoy...,train,fear,1


In [36]:
#Dividing the dataframe into a train and a test sections
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(twitter.text,twitter.emotion,
                                                test_size=0.2, random_state = 42)

In [14]:
%%time
import nltk
from sklearn.feature_extraction.text import CountVectorizer

BOW = CountVectorizer(tokenizer=nltk.word_tokenize, max_features=35000) #max_features=500

BOW.fit(x_train)

C:\Users\User\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


Wall time: 19.3 s


CountVectorizer(max_features=5000,
                tokenizer=<function word_tokenize at 0x0000021528110310>)

In [15]:
feature_names = BOW.get_feature_names()

In [16]:
"😕" in feature_names

True

In [17]:
x_train

255439    have you ever thought about the fact that it’s...
59813     doomseee doomseee you misspelled doomsee unles...
101110    cultural appropriation one word yoga culturala...
212739    jancello carnegiehall  billmurray hero kinda g...
43393                         musta been a sale on denim 😂 
                                ...                        
119879        even the halftime performance lacked offense 
259178    closed sell  lots eurusd  for  pips total for ...
131932    the patriots are no better than the jets right...
146867                 just so much bored  frustrated and  
121958    inthenameofindia congress decides to play divi...
Name: text, Length: 254130, dtype: object

In [18]:
import keras

x_train = BOW.transform(x_train)
#y_train = y_train

x_test = BOW.transform(x_test)
#y_test = y_test

print('x_train.shape: ', x_train.shape)
print('y_train.shape: ', y_train.shape)
print('x_test.shape: ', x_test.shape)
print('y_test.shape: ', y_test.shape)

x_train.shape:  (254130, 5000)
y_train.shape:  (254130,)
x_test.shape:  (63533, 5000)
y_test.shape:  (63533,)


In [19]:
y_test

33625            anger
32213              joy
71328         surprise
10175            trust
156317           trust
              ...     
225865         disgust
260173         disgust
230576    anticipation
222294        surprise
83967            trust
Name: emotion, Length: 63533, dtype: object

In [20]:
type(x_train)

scipy.sparse.csr.csr_matrix

In [21]:
y_train

255439    surprise
59813      sadness
101110     disgust
212739       trust
43393         fear
            ...   
119879        fear
259178         joy
131932     disgust
146867       anger
121958     disgust
Name: emotion, Length: 254130, dtype: object

In [22]:
## deal with label (string -> one-hot)

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

def label_encode(le, labels):
    enc = le.transform(labels)
    #return keras.utils.to_categorical(enc)
    return keras.utils.np_utils.to_categorical(enc)   #Allison/Moo said so, because of version

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)
y_test = label_encode(label_encoder, y_test)

print('\n\n## After convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)


check label:  ['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']

## Before convert
y_train[0:4]:
 255439    surprise
59813      sadness
101110     disgust
212739       trust
Name: emotion, dtype: object

y_train.shape:  (254130,)
y_test.shape:  (63533,)


## After convert
y_train[0:4]:
 [[0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]

y_train.shape:  (254130, 8)
y_test.shape:  (63533, 8)


In [23]:
# I/O check
input_shape = x_train.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

input_shape:  5000
output_shape:  8


In [24]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax

# input layer
model_input = Input(shape=(input_shape, ))  # 10000
X = model_input

# 1st hidden layer
X_W1 = Dense(units=64)(X)  # 64,32
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=64)(H1)  # 64,32
H2 = ReLU()(H1_W2)

# output layer
H2_W3 = Dense(units=output_shape)(H2)  # 4
H3 = Softmax()(H2_W3)

model_output = H3

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 5000)]            0         
                                                                 
 dense (Dense)               (None, 64)                320064    
                                                                 
 re_lu (ReLU)                (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 re_lu_1 (ReLU)              (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 8)                 520       
                                                                 
 softmax (Softmax)           (None, 8)                 0     

In [25]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger('logs/training_log.csv')

# training setting
epochs = 25
batch_size = 64

# training!
history = model.fit(x_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=[csv_logger],
                    validation_data = (x_test, y_test))
print('training finish')

Epoch 1/25


C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/dense/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/dense/embedding_lookup_sparse/Reshape:0", shape=(None, 64), dtype=float32), dense_shape=Tensor("gradient_tape/model/dense/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


3971/3971 [==============================] - 8s 2ms/step - loss: 1.6065 - accuracy: 0.4082 - val_loss: 1.5472 - val_accuracy: 0.4323
Epoch 2/25
3971/3971 [==============================] - 8s 2ms/step - loss: 1.4719 - accuracy: 0.4626 - val_loss: 1.5462 - val_accuracy: 0.4338
Epoch 3/25
3971/3971 [==============================] - 8s 2ms/step - loss: 1.3927 - accuracy: 0.4946 - val_loss: 1.5623 - val_accuracy: 0.4291
Epoch 4/25
3971/3971 [==============================] - 8s 2ms/step - loss: 1.3176 - accuracy: 0.5256 - val_loss: 1.6047 - val_accuracy: 0.4234
Epoch 5/25
3971/3971 [==============================] - 8s 2ms/step - loss: 1.2470 - accuracy: 0.5535 - val_loss: 1.6547 - val_accuracy: 0.4211
Epoch 6/25
3971/3971 [==============================] - 8s 2ms/step - loss: 1.1835 - accuracy: 0.5778 - val_loss: 1.7112 - val_accuracy: 0.4165
Epoch 7/25
3971/3971 [==============================] - 8s 2ms/step - loss: 1.1265 - accuracy: 0.6001 - val_loss: 1.7761 - val_accuracy: 0.4103
Epo

In [26]:
## predict
pred_result = model.predict(x_test, batch_size=128) #128
pred_result[:5]

array([[7.3125936e-02, 8.4794708e-02, 1.6015072e-01, 2.0436341e-01,
        1.8838915e-01, 2.2306383e-01, 1.3508905e-02, 5.2603323e-02],
       [2.5909230e-02, 2.4049428e-01, 4.8615481e-03, 1.5954448e-01,
        2.5047976e-01, 8.1039332e-02, 6.4712211e-02, 1.7295921e-01],
       [4.1640028e-01, 2.7003378e-01, 5.4407746e-02, 1.4857202e-03,
        7.2487915e-04, 6.4973631e-03, 2.4792923e-01, 2.5209750e-03],
       [5.1111871e-01, 1.8782986e-02, 7.4551383e-05, 2.0236896e-01,
        9.5500881e-03, 6.3989472e-05, 5.4908602e-04, 2.5749168e-01],
       [5.0795225e-06, 4.1107032e-02, 2.4736724e-05, 1.1050173e-04,
        4.0103678e-05, 9.2584064e-04, 3.5749935e-02, 9.2203683e-01]],
      dtype=float32)

In [27]:
pred_result = label_decode(label_encoder, pred_result)
pred_result[:5]

array(['sadness', 'joy', 'anger', 'anger', 'trust'], dtype=object)

In [28]:
from sklearn.metrics import accuracy_score

print('testing accuracy: {}'.format(round(accuracy_score(label_decode(label_encoder, y_test), pred_result), 2)))

testing accuracy: 0.38


In [29]:
#Let's take a look at the training log
training_log = pd.DataFrame()
training_log = pd.read_csv("logs/training_log.csv")
training_log

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.408248,1.606529,0.432326,1.547189
1,1,0.462649,1.471947,0.433838,1.546177
2,2,0.494641,1.392706,0.429147,1.562348
3,3,0.525609,1.317649,0.423433,1.604676
4,4,0.553488,1.247015,0.421135,1.654745
5,5,0.577846,1.183496,0.416539,1.711175
6,6,0.600091,1.126486,0.410259,1.776064
7,7,0.621371,1.074741,0.405789,1.862427
8,8,0.637764,1.028536,0.402122,1.926172
9,9,0.653268,0.987071,0.400138,2.020573


### Predicting on test data

In [30]:
twitter_test_data = pd.read_csv("twitter_test_data.csv")

In [31]:
x_test_twitter = BOW.transform(twitter_test_data['text'])

pred_result_test_data = model.predict(x_test_twitter, batch_size=128)

print('x_test.shape: ', x_test_twitter.shape)


x_test.shape:  (411972, 5000)


In [32]:
len(pred_result_test_data)

411972

In [33]:
pred_result_test_data = label_decode(label_encoder, pred_result_test_data)
pred_result_test_data[:5]

array(['trust', 'anticipation', 'anticipation', 'joy', 'anticipation'],
      dtype=object)

In [34]:
upload_df = pd.DataFrame()
upload_df["id"] = twitter_test_data["tweet_id"]
upload_df["emotion"] = pred_result_test_data
upload_df

,id,emotion
0,0x28b412,trust
1,0x2de201,anticipation
2,0x218443,anticipation
3,0x2939d5,joy
4,0x26289a,anticipation
...,...,...
411967,0x2913b4,anticipation
411968,0x2a980e,joy
411969,0x316b80,anticipation
411970,0x29d0cb,anger


In [35]:
upload_df.emotion.nunique()

8

In [ ]:
upload_df.to_csv("predictions/BOWKeras.csv",index=False)

# look for stemmer: going -> go 